# GPT4All

[GitHub:nomic-ai/gpt4all](https://github.com/nomic-ai/gpt4all) an ecosystem of open-source chatbots trained on a massive collections of clean assistant data including code, stories and dialogue.

This example goes over how to use LangChain to interact with `GPT4All` models.

In [1]:
%pip install gpt4all > /dev/null

Note: you may need to restart the kernel to use updated packages.


In [2]:
from langchain import PromptTemplate, LLMChain
from langchain.llms import GPT4All
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [3]:
template = """Question: {question}

Answer: Let's think step by step."""

prompt = PromptTemplate(template=template, input_variables=["question"])

### Specify Model

To run locally, download a compatible ggml-formatted model. 
 
**Download option 1**: The [gpt4all page](https://gpt4all.io/index.html) has a useful `Model Explorer` section:

* Select a model of interest
* Download using the UI and move the `.bin` to the `local_path` (noted below)

For more info, visit https://github.com/nomic-ai/gpt4all.

--- 

**Download option 2**: Uncomment the below block to download a model. 

* You may want to update `url` to a new version, whih can be browsed using the [gpt4all page](https://gpt4all.io/index.html).

In [ ]:
local_path = (
    "./models/ggml-gpt4all-l13b-snoozy.bin"  # replace with your desired local file path
)

# import requests

# from pathlib import Path
# from tqdm import tqdm

# Path(local_path).parent.mkdir(parents=True, exist_ok=True)

# # Example model. Check https://github.com/nomic-ai/gpt4all for the latest models.
# url = 'http://gpt4all.io/models/ggml-gpt4all-l13b-snoozy.bin'

# # send a GET request to the URL to download the file. Stream since it's large
# response = requests.get(url, stream=True)

# # open the file in binary mode and write the contents of the response to it in chunks
# # This is a large file, so be prepared to wait.
# with open(local_path, 'wb') as f:
#     for chunk in tqdm(response.iter_content(chunk_size=8192)):
#         if chunk:
#             f.write(chunk)

### Supporting token streaming in gpt4all using langchain

There are two different ways where we can do streaming of tokens coming out as the response from gpt4all models.

**Option 1**: Using custom callbacks provided by langchain. 

We use langchain's `StreamingStdOutCallbackHandler` callback to enable streaming of tokens. Below code shows, how we can
enable token streaming by adding `StreamingStdOutCallbackHandler` callback while initializing a gpt4all model through langchain.

In [ ]:
# Callbacks support token-wise streaming
callbacks = [StreamingStdOutCallbackHandler()]

# Verbose is required to pass to the callback manager
llm = GPT4All(model=local_path, callbacks=callbacks, verbose=True)

# If you want to use a custom model add the backend parameter
# Check https://docs.gpt4all.io/gpt4all_python.html for supported backends
# Do not initialize the llm twice. Either initialize the LLM without specifying backends

llm = GPT4All(model=local_path, backend="gptj", callbacks=callbacks, verbose=True)

In [ ]:
# create the LLM chain by chaining the prompt Template and the LLM togather
# and run the chain

llm_chain = LLMChain(prompt=prompt, llm=llm)

question = "What NFL team won the Super Bowl in the year Justin Bieber was born?"
llm_chain.run(question)

---

**Option 2**: Enabling streaming using without using callbacks. 

This option is only supported for gpt4all version >= `1.0.3`. If you want to upgrade your gpt4all package just use this command:

```bash
pip install --upgrade gpt4all
```

Once upgrading is finished then you can easily allow token streaming by just enabling `streaming` parameter as `True`
while initializing the LLM. Below code shows how that is done.

In [ ]:
# initialize your gpt4all model with allow_streaming = True

llm = GPT4All(model=local_path, backend="gptj", streaming=True)

# build the chain using LLMChain and run it with the question

llm_chain = LLMChain(prompt=prompt, llm=llm)

question = "What NFL team won the Super Bowl in the year Justin Bieber was born?"
llm_response = llm_chain.run(question)